# Pinecone 환경 설정 확인

In [6]:
from dotenv import load_dotenv
import os

import pinecone
from pinecone import Pinecone as PineconeClient
from pinecone import ServerlessSpec

In [7]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_environment = os.getenv("PINECONE_ENVIRONMENT")
print(pinecone_api_key[:10])
print(pinecone_environment[:10])

pcsk_2jise
us-east-1-


In [8]:
pc = PineconeClient(api_key=pinecone_api_key, environment =pinecone_environment)

In [12]:
#서비스 서버에 없으면 인덱스 생성
index_name = "example-index"
embedding_dim = 1536 #text-embedding-3-small의 벡터차원
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name, 
        dimension=embedding_dim,
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )

In [14]:
#서비스 서버의 인덱스와 연결
index = pc.Index(index_name)

In [15]:
# OpenAI의 임베딩 모델 설정: text-embedding-3-small
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [17]:
from langchain_core.documents import Document

# 예시 문서 생성 (내용과 메타데이터)
doc1 = Document(page_content="Building an exciting new project with LangChain - come check it out!",
                metadata={"source": "tweet"})

doc2 = Document(page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
                metadata={"source": "news"})

doc3 = Document(page_content="LangGraph is the best framework for building stateful, agentic applications!",
                metadata={"source": "tweet"})


In [19]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [20]:
vector_store.add_documents([doc1, doc2, doc3])

['474a4424-6395-4caf-ba34-e352b42e13de',
 '870845b4-cdea-4b07-b1e7-619dd7c35a25',
 '7d6ebe46-96c5-428f-8899-55983d9667ec']

In [21]:
#쿼리 해보기
query1 = "LangChain은 LLM을 쉽게 작업할 수 있도록 추상화 제공"
results1 = vector_store.similarity_search(query=query1, k=2)
print(f"쿼리 : {query1}")
for result in results1:
    print(f"* {result.page_content} (source: {result.metadata['source']})")


쿼리 : LangChain은 LLM을 쉽게 작업할 수 있도록 추상화 제공
* Building an exciting new project with LangChain - come check it out! (source: tweet)
* LangGraph is the best framework for building stateful, agentic applications! (source: tweet)


In [ ]:
results2 = vector_store.similarity_search(query=query1, k=2, filter={"source": "tweet"})
print(f"쿼리 : {query1}")
for result in results2:
    print(f"* {result.page_content} (source: {result.metadata['source']})")

쿼리 : LangChain은 LLM을 쉽게 작업할 수 있도록 추상화 제공
* Building an exciting new project with LangChain - come check it out! (source: tweet)
* LangGraph is the best framework for building stateful, agentic applications! (source: tweet)


In [28]:
query3 = "Will it be hot tomorrow"
results3 = vector_store.similarity_search_with_score(query=query3, k=3, filter={"source": "news"})
print(f"쿼리 : {query3}")
for result, score in results3:
    print(f"* SIM=({score:.4f}){result.page_content} (source: {result.metadata['source']})")

쿼리 : Will it be hot tomorrow
* SIM=(0.5440)The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. (source: news)
